In [7]:
import json
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

In [2]:
# --- US company list

response = requests.get('https://financialmodelingprep.com/api/v3/stock/list?apikey=f33b3631d5140a4f1c87e7f2eafd8fdd')
response.encoding = 'utf-8'
raw_list=response.json()
comp_list = [i['symbol'] for i in raw_list]
exchange = set([i['exchange'] for i in raw_list])
exchange_us = ['NASDAQ',
 'NASDAQ Global Market',
 'NYSE',
 'NYSE American',
 'Nasdaq',
 'Nasdaq Capital Market',
 'Nasdaq Global Market',
 'Nasdaq Global Select',
 'New York Stock Exchange',
 'New York Stock Exchange Arca']
us_list = [i['symbol'] for i in raw_list if i['exchange'] in exchange_us]

In [4]:
def file(n):
    with open(f'data/{n}.txt', encoding = 'utf-8') as f:
        data = json.load(f)
        f.close()
    return data

In [ ]:
asset = file('asset')
income = file('income')
cashflow = file('cashflow')
growth = file('growth')
ratio = file('ratio')
profile = file('profile')

In [5]:
profile = file('profile')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
def checker(n):
    temp = []
    temp2 = []
    for i in n:
        if i != None:
            for j in i:
                if j != None:
                    temp.append(j['period'])
                    temp2.append(j['date'][0:4])
    return Counter(temp), Counter(temp2)

checker(growth)


In [40]:
# --- Add a time series tag for quarter

def preprocessing(n: list):

    temp = []
    for i in n:
        if i != None:
            for j in i:
                if len(j['date']) == 10 and j['period'] != None and j['period'].startswith('Q') and len(j['period']) == 2:
                    year = int(j['date'][0:4])
                    quarter = int(j['period'][-1])
                    j['quarter'] = (year - 2001)*4 + quarter
                    temp.append(j)
                else:
                    i.remove(j)
    return temp

asset2 = preprocessing(asset)
income2 = preprocessing(income)
cashflow2 = preprocessing(cashflow)
growth2 = preprocessing(growth)
ratio2 = preprocessing(ratio)

    

In [32]:
# ---- After finishing: convert to dataframe

asset_df = pd.DataFrame(asset2)
asset_df.to_csv('data/asset_df.csv')
income_df = pd.DataFrame(income2)
income_df.to_csv('data/income_df.csv')
cashflow_df = pd.DataFrame(cashflow2)
cashflow_df.to_csv('data/cashflow_df.csv')    
growth_df = pd.DataFrame(growth2)
growth_df.to_csv('data/growth_df.csv')
ratio_df = pd.DataFrame(ratio2)
ratio_df.to_csv('data/ratio_df.csv')

In [41]:
def year(n):
    n['year'] = n['date'].str.slice(0,4)

for i in [asset_df, income_df, cashflow_df, growth_df, ratio_df]:
    year(i)

In [42]:
combined_df = pd.merge(asset_df, income_df, on=['symbol', 'quarter'], how='inner', left_index=True, right_index=True, suffixes=('', '_drop'))
combined_df = pd.merge(combined_df, cashflow_df, on=['symbol', 'quarter'], how='inner', left_index=True, right_index=True, suffixes=('', '_drop'))
combined_df = pd.merge(combined_df, growth_df, on=['symbol', 'quarter'], how='inner', left_index=True, right_index=True, suffixes=('', '_drop'))
combined_df = pd.merge(combined_df, ratio_df, on=['symbol', 'quarter'], how='inner', left_index=True, right_index=True, suffixes=('', '_drop'))

In [35]:
irrelevant = ['link','finalLink', 'goodwillAndIntangibleAssets', 'epsdiluted', 'weightedAverageShsOutDil','operatingCashFlow','capitalExpenditure']

In [43]:
combined_df.drop(columns=irrelevant, inplace=True)
combined_df.drop([i for i in combined_df.columns if 'drop' in i], axis=1, inplace=True)
combined_df['date'] = pd.to_datetime(combined_df['date'])
combined_df.head()


,date,symbol,reportedCurrency,fillingDate,acceptedDate,period,cashAndCashEquivalents,shortTermInvestments,cashAndShortTermInvestments,netReceivables,...,priceToSalesRatio,priceEarningsRatio,priceToFreeCashFlowsRatio,priceToOperatingCashFlowsRatio,priceCashFlowRatio,priceEarningsToGrowthRatio,priceSalesRatio,dividendYield,enterpriseValueMultiple,priceFairValue
0,2021-03-31,CMCSA,USD,2021-04-29,2021-04-29 15:22:23,Q1,14950000000,0,14950000000,1.098600e+10,...,9.131373,18.655678,47.049056,32.049931,32.049931,-17.455952,9.131373,0.004347,47.118783,2.683435
1,2020-12-31,CMCSA,USD,2021-02-04,2021-02-03 18:08:14,Q4,11740000000,0,11740000000,1.146600e+10,...,8.662711,17.753431,157.601046,47.605393,47.605393,0.263889,8.662711,0.004391,55.651505,2.616434
2,2020-09-30,CMCSA,USD,2020-10-29,2020-10-29 15:03:25,Q1,13707000000,0,13707000000,1.031000e+10,...,8.292809,26.217436,92.499786,40.499620,40.499620,-0.819628,8.292809,0.004997,48.797840,2.417833
3,2020-06-30,CMCSA,USD,2020-07-30 00:00:00,2020-07-30 18:31:48,Q2,13935000000,0,13935000000,1.022700e+10,...,7.572459,15.025172,30.100714,20.777607,20.777607,0.380639,7.572459,0.005853,34.187316,2.117924
4,2020-03-31,CMCSA,USD,2020-04-30 00:00:00,2020-04-30 15:24:15,Q1,8516000000,0,8516000000,1.080000e+10,...,5.965368,18.483054,47.739087,27.254887,27.254887,-0.577117,5.965368,0.006155,31.270757,1.917452


In [44]:
# --- Validation of data

test = combined_df[['date','symbol','year','period','quarter']]
test.head()

,date,symbol,year,period,quarter
0,2021-03-31,CMCSA,2021,Q1,81
1,2020-12-31,CMCSA,2020,Q4,80
2,2020-09-30,CMCSA,2020,Q1,77
3,2020-06-30,CMCSA,2020,Q2,78
4,2020-03-31,CMCSA,2020,Q1,77


In [53]:
test['Quarter'] = test['date'].apply(lambda x: int(int(x.split('-')[1])/3))
test['Validate'] = (test['period'].apply(lambda x: int(x[-1])) == test['Quarter'])
test.groupby('Validate')['Validate'].count()

<ipython-input-53-fcee33656cba>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Quarter'] = test['date'].apply(lambda x: int(int(x.split('-')[1])/3))
<ipython-input-53-fcee33656cba>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Validate'] = (test['period'].apply(lambda x: int(x[-1])) == test['Quarter'])


Validate
False     76111
True     220628
Name: Validate, dtype: int64

0        2021-03-31
1        2020-12-31
2        2020-09-30
3        2020-06-30
4        2020-03-31
            ...    
296734   2017-09-30
296735   2017-06-30
296736   2017-03-31
296737   2016-12-31
296738   2016-09-30
Name: date, Length: 296739, dtype: datetime64[ns]

In [70]:
# --- Remaking correct quarter number

combined_df['period'] = (pd.DatetimeIndex(combined_df['date']).month/3).astype(int)
combined_df['quarter'] = (pd.DatetimeIndex(combined_df['date']).year - 2001)*4 + combined_df['period']
combined_df['quarter']


0         81
1         80
2         79
3         78
4         77
          ..
296734    67
296735    66
296736    65
296737    64
296738    63
Name: quarter, Length: 296739, dtype: int64

In [71]:
combined_df.to_csv('data/combined.csv')